In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [ ]:
base_trip = pd.read_csv(f"{base_dir}/final_trips.csv", 
                        index_col='trip_id', 
                        usecols=['trip_id','trip_count','trip_num','origin','destination','depart'])
base_trip_idx = base_trip.index
build_trip = pd.read_csv(f"{build_dir}/final_trips.csv", 
                         index_col='trip_id', 
                         usecols=['trip_id','trip_count','trip_num','origin','destination','depart'])


In [ ]:
print(f"""Filtering trips by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal trips in base: {len(base_trip)}\tbuild: {len(build_trip)}\tdiff: {len(build_trip)-len(base_trip)}""")

base_trip = base_trip[multi_filter_mazs([base_trip.origin, base_trip.destination])] # base tour in the filtered set

build_trip = build_trip[

    (build_trip.index.isin(base_trip_idx) # originally existed in base
    & build_trip.index.isin(base_trip.index) )    # and was in the current set for the base
 
    |     # OR
    ((~build_trip.index.isin(base_trip_idx)) # is a new tour 
     & multi_filter_mazs([build_trip.origin, build_trip.destination])) # and it's in this set
     
    ]

print(f"After filtering, trips in base: {len(build_trip)}\tbuild: {len(build_trip)}\tdiff: {len(build_trip)-len(base_trip)}")

## Trip Purpose

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Trip Destination

In [ ]:
df = base_trip[base_trip.trip_count != base_trip.trip_num].merge(build_trip[build_trip.trip_count != build_trip.trip_num],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))


In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Trip Scheduling

In [ ]:
# translate departure to time period index
base_trip['depart_period'] = base_trip.depart.apply(vh.get_time_period_index)
build_trip['depart_period'] = build_trip.depart.apply(vh.get_time_period_index)

# join base and build trips on trip_id
# compare trips that are in both base and build
df = base_trip.merge(
    build_trip, 
    how='inner', 
    left_index=True, 
    right_index=True,
    suffixes=('_base', '_build')
)

# get the difference in departure time
df['depart_bin_difference'] = df.depart_build - df.depart_base
# get the difference in departure time period
df['depart_period_difference'] = df.depart_period_build - df.depart_period_base

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

for key in ['bin', 'period']:
    purpose_df = df
    display(Markdown(f"### Trip Departure Time {key.capitalize()} Difference"))
    purpose_df = purpose_df.groupby(f'depart_{key}_difference').size().reset_index()
    purpose_df.columns = [f'depart_{key}_difference', 'count']
    purpose_df = purpose_df.sort_values(by=f'depart_{key}_difference')

    fig = vh.create_bar_chart(
        source_data=purpose_df,
        source=f'depart_{key}_difference',
        col=f'depart_{key}_difference',
        plot_col='count',
        # title=f"{purpose.capitalize()} Tour {metric.capitalize()} Time Difference",
    )
    fig.show()
    display(Markdown(" "))

## Trip Mode Choice

In [ ]:
print("Lorem ipsum") # Placeholder for CS